In [ ]:
from pathlib import Path
from newclid import GeometricSolverBuilder
from newclid.problem import PredicateConstruction, ProblemSetup
from newclid.jgex.problem_builder import JGEXProblemBuilder
from newclid.jgex.formulation import JGEXFormulation
from newclid.api import PythonDefault

JGEX_PROBLEM = """B C = segment B C;
D = on_tline D C B C, on_circle D C B;
E = on_line E C D;
A = on_pline A D B C, on_circle A D E;
F = intersection_lt F A B E A B;
G = intersection_ll G B D A E;
H = intersection_ll H B D E F
""".strip().replace('\n', ' ')
goals = [PredicateConstruction.from_str("eqratio D H E H D B E B")]


# Build a JGEX formulation into a ProblemSetup
jgex_problem = JGEXFormulation.from_text(JGEX_PROBLEM)
problem_builder = JGEXProblemBuilder(problem=jgex_problem, rng=0)
problem_setup = problem_builder.build()
problem = ProblemSetup(name=problem_setup.name,
                        points=problem_setup.points,
                        assumptions=problem_setup.assumptions,
                        goals=tuple(goals),
                        )
print(problem.pretty_str())
# Build a solver from the ProblemSetup
solver = GeometricSolverBuilder(api_default=PythonDefault(use_sympy_ar=True)
                                ).build(problem)

# Run deduction + algebraic reasoning
success = solver.run()
print("solved: ", success)

# Print proof (text form)
if success:
    solver.write_all_outputs(Path('out'), )
    proof = solver.proof()
    print(proof)

# Optionally dump figure/proof artifacts
# solver.write_all_outputs("out_trapezoid_eqratio", jgex_problem=problem_builder.jgex_problem)

Built the problem problem with assumptions:
coll A B F
coll A E G
coll B D G
coll B D H
coll C D E
coll E F H
cong C D B C
cong D E A D
diff B C
diff B C
diff B C
diff C D
diff D E
ncoll A B D E
ncoll A B E
ncoll B C D
ncoll B D E F
npara B D A E
npara B D E F
nperp A B A B
para B C A D
perp A B E F
perp B C C D

and goals:
eqratio B D B E D H E H

Using points:
A (1.1561934055334837, -1.4927309808941782)
B (0.2739233746429086, -0.4604265724722594)
C (-0.9180529521276106, -0.9669447289429418)
D (-0.4115347956569282, -2.1589210557134613)
E (-1.0777248704762106, -0.5911928545230505)
F (-0.23204119209588506, 0.1315797685144301)
G (0.0392342675286359, -1.0419619177086141)
H (0.9050296630782655, 1.1033895664139326)



TypeError: unhashable type: 'LineMerge'